# Data Augmentation

좋은 품질의 데이터가 많을수록 모델의 정확도가 향상된다.   
신경망을 학습시키기 위한 좋은 품질의 데이터가 부족할 때 기존의 데이터를 변형하여 데이터의 숫자를 증가시킬 수 있다.  
Data Augmentation을 사용하면 기존의 모델보다 약간의 정확도를 높일 수 있지 않을까?


In [ ]:
import pandas as pd
import numpy as np
import random

from transformers import AutoTokenizer

In [2]:
train = pd.read_csv("data/train_data.csv")

In [3]:
train.head()

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4


텍스트 데이터를 augmentation하는 방법에는 다음과 같은 방법이 있다.  
1. SR ( Synonynm Replace ) : 특정 단어를 유의어로 교체한다. 
2. RI ( Random Insertion ) : 특정 단어를 임의로 넣는다.
3. RD ( Random Deletion ) : 특정 단어를 임의로 제거한다. 
4. RS ( Random Swap ) : 특정 단어들의 위치를 임의로 바꾼다.

구글링을 하다가 텍스트 데이터 augmentation을 쉽게 적용할 수 있도록 정리된 코드를 발견하여 참고했다.  
https://github.com/catSirup/KorEDA

SR과 RI 같은 경우 RD와 RS에 비해서 문장의 의미가 바뀔 가능성이 높기 때문에.  
이번 대회에서는 RD와 RS를 사용하여 테스트를 했다.  
문장을 토큰화하기 위해서 Klue/bert-base의 Tokenizer를 사용했다. 

In [9]:
model_checkpoint = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [29]:
test_sentence = "안녕하세요."
print(tokenizer.tokenize(test_sentence))
test_token = ['안녕', '##하', '##세요', '.']
print(tokenizer.convert_tokens_to_string(test_token))

['안녕', '##하', '##세요', '.']
안녕하세요.


## Random Deletion

In [12]:
train["tokenized"] = [tokenizer.tokenize(sentence) for sentence in train["title"]]

In [13]:
train.head()

,index,title,topic_idx,tokenized
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4,"[인천, ##→, ##핀, ##란드, 항공기, 결, ##항, …, 휴가철, 여행객,..."
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4,"[실리콘밸리, 넘어서, ##겠다, …, 구글, 15, ##조, ##원, 들여, 美,..."
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4,"[이란, 외무, 긴장, ##완, ##화, 해결책, ##은, 미국, ##이, 경제, ..."
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4,"[NYT, 클린턴, 측근, 韓, 기업, 특수, ##관계, 조명, …, 공과, 사, ..."
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4,"[시진핑, 트럼프, ##에, 중, ##미, 무역, ##협, ##상, 조속, 타결, 희망]"


random_deletion 함수를 정의한다.  
p는 문장에서 임의의 단어를 삭제할 확률이다.

In [15]:
def random_deletion(words, p):
    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    return new_words

In [16]:
rd = [random_deletion(tokenized,0.2) for tokenized in train.tokenized]

임의의 단어들이 삭제가 되었다.

In [38]:
rd[:3]

[['인천', '##→', '##핀', '##란드', '항공기', '##항', '분통'],
 ['실리콘밸리', '넘어서', '##조', '들여', '전역', '거점'],
 ['외무', '긴장', '해결책', '##은', '미국', '##이', '경제', '멈추', '##는', '것']]

Augmentation을 수행한 데이터들은 저장해놓고 다음 학습에서 사용하자.

In [32]:
rd_augmentation = pd.DataFrame({'augmented' : rd, 'topic_idx': train.topic_idx})
rd_augmentation.to_csv("data/train_rd_augmentation.csv")

## Random Swap

In [33]:
train.head()

,index,title,topic_idx,tokenized
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4,"[인천, ##→, ##핀, ##란드, 항공기, 결, ##항, …, 휴가철, 여행객,..."
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4,"[실리콘밸리, 넘어서, ##겠다, …, 구글, 15, ##조, ##원, 들여, 美,..."
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4,"[이란, 외무, 긴장, ##완, ##화, 해결책, ##은, 미국, ##이, 경제, ..."
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4,"[NYT, 클린턴, 측근, 韓, 기업, 특수, ##관계, 조명, …, 공과, 사, ..."
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4,"[시진핑, 트럼프, ##에, 중, ##미, 무역, ##협, ##상, 조속, 타결, 희망]"


random_swap 함수를 정의한다.  
p는 문장에서 임의의 단어를 교체할 확률이다.

In [34]:
def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)

    return new_words

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0

    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words

아까 tokenize한 데이터들을 다시 사용하여 RS를 수행한다.

In [35]:
rs = [random_swap(tokenized,2) for tokenized in train.tokenized]

In [37]:
rs[:3]

[['인천', '휴가철', '##핀', '##→', '항공기', '결', '##항', '…', '##란드', '여행객', '분통'],
 ['실리콘밸리',
  '넘어서',
  '##겠다',
  '##조',
  '구글',
  '15',
  '…',
  '##원',
  '들여',
  '美',
  '거점',
  '전역',
  '##화'],
 ['이란',
  '외무',
  '긴장',
  '##완',
  '##는',
  '해결책',
  '##은',
  '미국',
  '##이',
  '경제',
  '멈추',
  '##전쟁',
  '##화',
  '것']]

이 데이터도 나중에 학습에 사용하도록 한다. 

In [39]:
rs_augmentation = pd.DataFrame({'augmented' : rd, 'topic_idx': train.topic_idx})
rs_augmentation.to_csv("data/train_rs_augmentation.csv")